# プロジェクト概要
Snowpark for Python、Snowpark ML、Streamlit を使用して、検索、動画、ソーシャルメディア、電子メールなどの複数のチャネルで変動する広告費予算の将来の ROI（Return On Investment）を予測する線形回帰モデルを学習するためのデータ分析とデータ準備タスクを実行します。

セッションの最後には、さまざまな広告費予算の ROI を視覚化するインタラクティブなウェブアプリケーションがデプロイされます。

Snowpark for Python および Snowpark ML は Amazon SageMaker Studio、Streamlit は Streamlit in Snowflake を用いて実行します。

***前提条件***: このノートブックを進める前に、まず次のノートブックを正常に実行し終えている必要があります [Snowpark_For_Python_DE.ipynb](Snowpark_For_Python_DE.ipynb)

### 機械学習
このノートブックでは、Snowpark for Python を使用した Snowflake での機械学習に焦点を当てます。

* Snowflake へのセキュアな接続の確立
* 特徴量とターゲットを Snowflake テーブルから Snowpark DataFrame にロード
* モデルトレーニングのための特徴量の準備
* Snowflake で Snowpark ML を使用して ML モデルをトレーニングし、モデルを Snowflake ステージにアップロード
* 新しいデータポイントに対する推論のために、Python のスカラー関数とベクトル化ユーザ定義関数（UDF）を作成
  *  *Note: Scalar UDF は Streamlit Apps [Snowpark_Streamlit_Revenue_Prediction_SiS.py](Snowpark_Streamlit_Revenue_Prediction_SiS.py) から呼び出されます。*

### ライブラリのインポート

In [ ]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import Variant
from snowflake.snowpark.functions import udf,sum,col,array_construct,month,year,call_udf,lit
from snowflake.snowpark.version import VERSION

# Snowpark ML
from snowflake.ml.modeling.compose import ColumnTransformer
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import PolynomialFeatures, StandardScaler
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.modeling.model_selection import GridSearchCV

# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

### Snowflake へのセキュアな接続の確立

Snowpark Python API を使用すると、Snowflake と Notebook 間のセキュアな接続を素早く簡単に確立できます。

TIP: [Session](https://docs.snowflake.com/developer-guide/snowpark/reference/python/session.html) オブジェクトについて

In [ ]:
# Snowflake Session オブジェクトの作成
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# 環境の詳細（connections.json の編集内容が適切に反映されているか確認してください）
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### 特徴量とターゲット

特徴量とモデル学習対象を保存するために、以下のアクションを実行します。

* 欠損値のある行の削除
* モデリングに不要な列の除外
* MARKETING_BUDGETS_FEATURES という Snowflake テーブルに特徴量を保存

In [ ]:
# データロード
snow_df_spend_and_revenue_per_month = session.table('spend_and_revenue_per_month')

# 欠損値のある行の削除
snow_df_spend_and_revenue_per_month = snow_df_spend_and_revenue_per_month.dropna()

# モデリングに不要な列の除外
snow_df_spend_and_revenue_per_month = snow_df_spend_and_revenue_per_month.drop(['YEAR','MONTH'])

# MARKETING_BUDGETS_FEATURES という Snowflake テーブルに特徴量を保存
snow_df_spend_and_revenue_per_month.write.mode('overwrite').save_as_table('MARKETING_BUDGETS_FEATURES')
snow_df_spend_and_revenue_per_month.show()

### Snowflake で Snowpark ML を使用して ML モデルをトレーニング

Snowpark ML は、SDK と基盤となるインフラストラクチャ機械学習を含むツールのセットです。Snowpark ML を使用すると、単一の SDK を使用して、データの前処理、ML モデルのトレーニングのすべてを Snowflake 内で行うことができ、機械学習ワークフローのすべての段階で、Snowflake の実証済みのパフォーマンス、スケーラビリティ、安定性、ガバナンスの恩恵を受けることができます。

In [ ]:
CROSS_VALIDATION_FOLDS = 10
POLYNOMIAL_FEATURES_DEGREE = 2

# Change session param
session.sql("alter session set language = 'en'").collect()

# トレーニングおよびテスト用 Snowpark DataDrames を作成
train_df, test_df = session.table("MARKETING_BUDGETS_FEATURES").random_split(weights=[0.8, 0.2], seed=0)

# 数値カラムの前処理
# PolynomialFeatures と StandardScaler の前処理を数値カラムに適用
# NOTE: 次数が高くなると過学習しやすくなる
numeric_features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
numeric_transformer = Pipeline(steps=[('poly',PolynomialFeatures(degree = POLYNOMIAL_FEATURES_DEGREE)),('scaler', StandardScaler())])

# ColumnTransformer を使って前処理のステップを結合
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

# 次に前処理した特徴量をアルゴリズムと統合し、モデルを構築
pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LinearRegression())])
parameteres = {}

# GridSearch を使って number_of_folds に基づいて最適なモデルを探索
model = GridSearchCV(
    estimator=pipeline,
    param_grid=parameteres,
    cv=CROSS_VALIDATION_FOLDS,
    label_cols=["REVENUE"],
    output_cols=["PREDICTED_REVENUE"],
    verbose=2
)

# 学習と評価
model.fit(train_df)
train_r2_score = model.score(train_df)
test_r2_score = model.score(test_df)

# トレーニングおよびテスト用データに対する決定係数 R2 乗値
print(f"R2 score on Train : {train_r2_score}")
print(f"R2 score on Test  : {test_r2_score}")

#### トレーニングしたモデルを Snowflake ステージに保存

In [ ]:
import os
from joblib import dump

# SKLearn オブジェクトに抽出 
sk_model = model.to_sklearn()

model_output_dir = '/tmp'
model_file = os.path.join(model_output_dir, 'model.joblib')
dump(sk_model, model_file)
session.file.put(model_file,"models",overwrite=True)

### >>>>>>>>>> *Snowsight でクエリ履歴を調べてみましょう* <<<<<<<<<<

### 推論用のスカラーユーザ定義関数 (UDF) の作成

このモデルを推論用にデプロイするために、**Snowpark Python UDF を作成して登録し、学習済みモデルを依存関係**として追加しましょう。一度登録すれば、データを渡して関数を呼び出すだけで、新しい予測を得ることができます。

*注：スカラー UDF は 1 つの行/データポイントのセットで動作し、リアルタイムのオンライン推論に最適です。そして、この UDF は Streamlit アプリから呼び出されます。[Snowpark_Streamlit_Revenue_Prediction_SiS.py](Snowpark_Streamlit_Revenue_Prediction_SiS.py) を参照してください。*

TIP: [Snowpark Python User-Defined Functions](https://docs.snowflake.com/en/developer-guide/snowpark/python/creating-udfs.html) についての詳細はこちら

In [ ]:
session.clear_imports()
session.clear_packages()

# 学習済みモデルと Python パッケージを Snowflake Anaconda チャンネルから UDF 依存としてサーバーサイドで利用できるように追加
session.add_import('@models/model.joblib.gz')
session.add_packages('pandas','joblib','scikit-learn==1.1.1')

@udf(name='predict_roi',session=session,replace=True,is_permanent=True,stage_location='@udfs')
def predict_roi(budget_allocations: list) -> float:
    import sys
    import pandas as pd
    from joblib import load
    import sklearn

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    
    model_file = import_dir + 'model.joblib.gz'
    model = load(model_file)
            
    features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
    df = pd.DataFrame([budget_allocations], columns=features)
    roi = abs(model.predict(df)[0])
    return roi

### 新しいデータに対する推論のためにスカラーユーザー定義関数（UDF）を呼出

UDFが登録されると、_call_udf()_ Snowpark Python関数を呼び出して新しいデータポイントを渡すだけで、新しい予測を取得することができます。

サンプルデータで SnowPark DataFrame を作成し、UDF を呼び出して新しい予測を取得してみましょう。

*Note: この UDF は Streamlit アプリからも呼び出されます。[Snowpark_Streamlit_Revenue_Prediction_SiS.py](Snowpark_Streamlit_Revenue_Prediction_SiS.py) を参照してください。

In [ ]:
test_df = session.create_dataframe([[250000,250000,200000,450000],[500000,500000,500000,500000],[8500,9500,2000,500]], 
                                    schema=['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL'])
test_df.select(
    'SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL', 
    call_udf("predict_roi", 
    array_construct(col("SEARCH_ENGINE"), col("SOCIAL_MEDIA"), col("VIDEO"), col("EMAIL"))).as_("PREDICTED_ROI")).show()

### バッチ API を使用した推論用ベクトル化ユーザ定義関数 (UDF) の作成

ここでは、Python UDF Batch API を活用して、Pandas Dataframe を入力とする **ベクトル化** UDF を作成します。これは、UDFの各呼び出しが、入力として 1 行を受け取るスカラー UDF と比較して、行のセット/バッチを受け取ることを意味します。

最初にヘルパー関数 _load_model()_ を作成します。これは **cachetools** を使用してモデルを一度だけロードするようにし、_batch_predict_roi()_ 関数が推論を行います。

*注：ベクトル化 UDF はバッチでのオフライン推論に最適です。

スカラー UDF よりもバッチ API を使用する利点：

* Python コードが行のバッチを効率的に操作する場合、パフォーマンスが向上する可能性があります
* Pandas DataFrames や Pandas 配列を操作するライブラリを呼び出す場合、変換ロジックが少なくて済みます

TIP: [Snowpark Python UDF Batch API](https://docs.snowflake.com/en/developer-guide/udf/python/udf-python-batch.html#getting-started-with-the-batch-api) の詳細についてはこちら

In [ ]:
session.clear_imports()
session.clear_packages()

import cachetools
from snowflake.snowpark.types import PandasSeries, PandasDataFrame

# 学習済みモデルと Python パッケージを Snowflake Anaconda チャンネルから UDF 依存としてサーバーサイドで利用できるように追加
session.add_import('@models/model.joblib.gz')
session.add_packages('pandas','joblib','scikit-learn','cachetools')

@cachetools.cached(cache={})
def load_model(filename):
    import joblib
    import sys
    import os

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m

@udf(name='batch_predict_roi',session=session,replace=True,is_permanent=True,stage_location='@udfs')
def batch_predict_roi(budget_allocations_df: PandasDataFrame[int, int, int, int]) -> PandasSeries[float]:
    import sklearn
    budget_allocations_df.columns = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
    model = load_model('model.joblib.gz')
    return abs(model.predict(budget_allocations_df))

### 新しいデータに対する推論のためにバッチ API を使用してベクトル化されたユーザー定義関数 (UDF) を呼出

*Note: Python UDF を使ったクエリの書き方を変更する必要はありません。すべてのバッチ処理は、あなた自身のコードではなく、UDF フレームワークによって処理されます。*

In [ ]:
test_df.select(
    'SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL', 
    call_udf("batch_predict_roi", 
    col("SEARCH_ENGINE"), col("SOCIAL_MEDIA"), col("VIDEO"), col("EMAIL")).as_("PREDICTED_ROI")).show()